## Used Car Price Prediction

### Import relevant libraries

In [37]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### Load and Explore the data

In [7]:
pd.set_option('display.max_rows', None)        # Show all rows
pd.set_option('display.max_columns', None)     # Show all columns
pd.set_option('display.width', None)           # Don’t wrap lines
pd.set_option('display.max_colwidth', None)    # Don’t truncate column content

In [2]:
df = pd.read_csv('data/used_cars.csv')

In [3]:
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"


In [4]:
df.shape

(4009, 12)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


### Data Cleaning

In [9]:
df.isnull().sum()

brand             0
model             0
model_year        0
milage            0
fuel_type       170
engine            0
transmission      0
ext_col           0
int_col           0
accident        113
clean_title     596
price             0
dtype: int64

fuel_type, accident, clean_title are three columns with null/missing values.

In [12]:
df['clean_title'].value_counts(dropna=False)

clean_title
Yes    3413
NaN     596
Name: count, dtype: int64

Upon inspection, the clean_title column contains only one unique value — "Yes" — with around 15% missing values. Since this feature shows no variance (i.e., all values are the same), it provides no predictive power to the model. Therefore, we can safely drop this column to simplify the dataset without losing any meaningful information.

In [13]:
df.drop(columns=['clean_title'], inplace=True)

In [14]:
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capability,6-Speed A/T,Black,Black,At least 1 accident or damage reported,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric Hybrid,7-Speed A/T,Black,Black,None reported,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,"$34,999"


In [15]:
# Mode imputation for fuel_type and accident columns

df['fuel_type'].fillna(df['fuel_type'].mode()[0], inplace=True)
df['accident'].fillna(df['accident'].mode()[0], inplace=True)

In [16]:
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capability,6-Speed A/T,Black,Black,At least 1 accident or damage reported,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric Hybrid,7-Speed A/T,Black,Black,None reported,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,"$34,999"


In [17]:
df.isnull().sum()

brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
price           0
dtype: int64

In [20]:
# check for duplicate rows
df.duplicated().sum()

0

In [34]:
# convert data types

df['milage'] = (
    df['milage']
    .str.replace('mi.', '', regex=False)  # remove the unit
    .str.replace(',', '', regex=False)   # remove commas
    .astype(float)                       # convert to float
)
df['price'] = df['price'].str.replace('$', '').str.replace(',', '').astype(float)

In [35]:
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,price
0,Ford,Utility Police Interceptor Base,2013,51000.0,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capability,6-Speed A/T,Black,Black,At least 1 accident or damage reported,10300.0
1,Hyundai,Palisade SEL,2021,34742.0,Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,38005.0
2,Lexus,RX 350 RX 350,2022,22372.0,Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,54598.0
3,INFINITI,Q50 Hybrid Sport,2015,88900.0,Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric Hybrid,7-Speed A/T,Black,Black,None reported,15500.0
4,Audi,Q3 45 S line Premium Plus,2021,9835.0,Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,34999.0


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   brand         4009 non-null   object 
 1   model         4009 non-null   object 
 2   model_year    4009 non-null   int64  
 3   milage        4009 non-null   float64
 4   fuel_type     4009 non-null   object 
 5   engine        4009 non-null   object 
 6   transmission  4009 non-null   object 
 7   ext_col       4009 non-null   object 
 8   int_col       4009 non-null   object 
 9   accident      4009 non-null   object 
 10  price         4009 non-null   float64
dtypes: float64(2), int64(1), object(8)
memory usage: 344.6+ KB


#### Engine feature extraction and cleaning

The engine column contains mixed, semi-structured information — including horsepower (HP), engine displacement (e.g., “3.5L”), cylinder type, and fuel type.
Among these, horsepower and engine capacity (in liters) are the most meaningful and consistently useful indicators of a vehicle’s performance and price.
Higher horsepower and larger engines generally correlate with higher vehicle prices.

In [38]:
df['engine']

0                        300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capability
1                                                        3.8L V6 24V GDI DOHC
2                                                              3.5 Liter DOHC
3                         354.0HP 3.5L V6 Cylinder Engine Gas/Electric Hybrid
4                                                  2.0L I4 16V GDI DOHC Turbo
5                                                                   2.4 Liter
6                                292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel
7                                282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel
8                               311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel
9                                 534.0HP Electric Motor Electric Fuel System
10                                                                         V6
11                              715.0HP 5.2L 12 Cylinder Engine Gasoline Fuel
12                      382.0HP 3.0L Straight 6 Cylinder Engine 

In [48]:
# Extract horsepower (e.g., "354.0HP" → 354.0)
df['horsepower'] = df['engine'].str.extract(r'(\d+\.?\d*)\s*HP', expand=False).astype(float)

# Extract engine capacity (e.g., "3.5L" or "3 L" → 3.5)
df['engine_capacity'] = df['engine'].str.extract(r'(\d+\.\d+|\d+)\s*[lL]', expand=False).astype(float)

In [49]:
df[['engine', 'horsepower', 'engine_capacity']].head(10)

,engine,horsepower,engine_capacity
0,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capability,300.0,3.7
1,3.8L V6 24V GDI DOHC,NaN,3.8
2,3.5 Liter DOHC,NaN,3.5
3,354.0HP 3.5L V6 Cylinder Engine Gas/Electric Hybrid,354.0,3.5
4,2.0L I4 16V GDI DOHC Turbo,NaN,2.0
5,2.4 Liter,NaN,2.4
6,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,292.0,2.0
7,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,282.0,4.4
8,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,311.0,3.5
9,534.0HP Electric Motor Electric Fuel System,534.0,NaN


Some vehicles are fully electric, while others are gas/electric hybrids.

In [50]:
# Detect electric and hybrid vehicles

df['is_electric'] = df['engine'].str.contains('Electric', case=False, na=False).astype(int)
df['is_hybrid'] = df['engine'].str.contains('Hybrid', case=False, na=False).astype(int)

In [52]:
# Fill missing horsepower by brand median
df['horsepower'] = df.groupby('brand')['horsepower'].transform(lambda x: x.fillna(x.median()))

# Fill missing engine_capacity by brand median
df['engine_capacity'] = df.groupby('brand')['engine_capacity'].transform(lambda x: x.fillna(x.median()))

# This respects brand-specific power characteristics — e.g., BMW cars typically have higher engine capacity than Suzuki.

In [53]:
# For fully electric (not hybrid) vehicles, set engine_capacity = 0
df.loc[(df['is_electric'] == 1) & (df['is_hybrid'] == 0), 'engine_capacity'] = 0

In [54]:
# Fill any remaining missing engine_capacity using the median of non-electric cars
non_electric_median = df.loc[df['is_electric'] == 0, 'engine_capacity'].median()
df['engine_capacity'].fillna(non_electric_median, inplace=True)

In [57]:
df[['horsepower', 'engine_capacity', 'is_electric', 'is_hybrid']].isnull().sum()

horsepower         2
engine_capacity    0
is_electric        0
is_hybrid          0
dtype: int64

In [58]:
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

In [59]:
df[['horsepower', 'engine_capacity', 'is_electric', 'is_hybrid']].isnull().sum()

horsepower         0
engine_capacity    0
is_electric        0
is_hybrid          0
dtype: int64

In [62]:
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,price,horsepower,engine_capacity,is_electric,is_hybrid
0,Ford,Utility Police Interceptor Base,2013,51000.0,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capability,6-Speed A/T,Black,Black,At least 1 accident or damage reported,10300.0,300.0,3.7,0,0
1,Hyundai,Palisade SEL,2021,34742.0,Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,38005.0,240.0,3.8,0,0
2,Lexus,RX 350 RX 350,2022,22372.0,Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,54598.0,301.0,3.5,0,0
3,INFINITI,Q50 Hybrid Sport,2015,88900.0,Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric Hybrid,7-Speed A/T,Black,Black,None reported,15500.0,354.0,3.5,1,1
4,Audi,Q3 45 S line Premium Plus,2021,9835.0,Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,34999.0,333.0,2.0,0,0


In [ ]:
# New feature -> Car Age

from datetime import datetime
current_year = datetime.now().year

df['car_age'] = current_year - df['model_year']